In [33]:
import pandas as pd
path_data = '../../../assets/data/'
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np

Population = pd.read_csv(path_data + 'Canada-Population_processed.csv')
Population = Population[['Geography', '%Growth']]
Houseprice = pd.read_csv(path_data + 'Canad-houseprice.csv')
Houseprice = Houseprice[['Area', 'y/y % change']]

# Joining DataFrames by Columns
Often, data about the same individuals is maintained in more than one table. For example, one university office might have data about each student's time to completion of degree, while another has data about the student's tuition and financial aid.

To understand the students' experience, it may be helpful to put the two datasets together. If the data are in two tables, each with one row per student, then we would want to put the columns together, making sure to match the rows so that each student's information remains on a single row.

Let us do this in the context of a simple example, and then use the method with a larger dataset.

The dataframe `cones` is one we have encountered earlier. Now suppose each flavor of ice cream comes with a rating that is in a separate table.

In [11]:
cones = pd.DataFrame(
    {'Flavor': ['strawberry', 'vanilla', 'chocolate', 'strawberry', 'chocolate'],
    'Price': [3.55, 4.75, 6.55, 5.25, 5.75]}
)
cones

,Flavor,Price
0,strawberry,3.55
1,vanilla,4.75
2,chocolate,6.55
3,strawberry,5.25
4,chocolate,5.75


In [12]:
ratings = pd.DataFrame(
    {'Kind': ['strawberry', 'chocolate', 'vanilla'],
    'Stars': [2.5, 3.5, 4]}
)
ratings

,Kind,Stars
0,strawberry,2.5
1,chocolate,3.5
2,vanilla,4.0


Each of the dataframes has a column that contains ice cream flavors: `cones` has the column `Flavor`, and `ratings` has the column `Kind`. The entries in these columns can be used to link the two dataframes.

The method `merge` creates a new dataframe in which each cone in the `cones` dataframe is augmented with the Stars information in the `ratings` table.  For each cone in `cones`, `merge` finds a row in `ratings` whose `Kind` matches the cone's `Flavor`. We have to tell `merge` to use those columns for matching.

In [13]:
rated = pd.merge(cones, ratings, left_on = 'Flavor', right_on= 'Kind')
rated

,Flavor,Price,Kind,Stars
0,strawberry,3.55,strawberry,2.5
1,strawberry,5.25,strawberry,2.5
2,vanilla,4.75,vanilla,4.0
3,chocolate,6.55,chocolate,3.5
4,chocolate,5.75,chocolate,3.5


Each cone now has not only its price but also the rating of its flavor.

In general, a call to `merge` that augments a dataframe (say `df1`) with information from another dataframe (say `df2`) looks like this:

    pd.merge(df1, df2, left_on = left_key, right_on= right_key)

The new dataframe `rated` allows us to work out the price per star, which you can think of as an informal measure of value. Low values are good – they mean that you are paying less for each rating star.

In [15]:
rated['$/Star'] = rated['Price'] / rated['Stars']
rated.sort_values('$/Star')

,Flavor,Price,Kind,Stars,$/Star
2,vanilla,4.75,vanilla,4.0,1.187500
0,strawberry,3.55,strawberry,2.5,1.420000
4,chocolate,5.75,chocolate,3.5,1.642857
3,chocolate,6.55,chocolate,3.5,1.871429
1,strawberry,5.25,strawberry,2.5,2.100000


Though strawberry has the lowest rating among the three flavors, the less expensive strawberry cone does well on this measure because it doesn't cost a lot per star.

**Side note.** Does the order we list the two dataframes matter? Let's try it.  As you see it, this changes the order that the columns appear in, and can potentially changes the order of the rows, but it doesn't make any fundamental difference.

In [16]:
rated = pd.merge(ratings, cones, right_on = 'Flavor', left_on= 'Kind')
rated

,Kind,Stars,Flavor,Price
0,strawberry,2.5,strawberry,3.55
1,strawberry,2.5,strawberry,5.25
2,chocolate,3.5,chocolate,6.55
3,chocolate,3.5,chocolate,5.75
4,vanilla,4.0,vanilla,4.75


Also note that the merge will only contain information about items that appear in both both dataframes, if the method of merge is not specified. Let's see an example. Suppose there is a dataframe of reviews of some ice cream cones, and we have found the average review for each flavor.

In [19]:
reviews = pd.DataFrame({
    'Flavor': ['vanilla', 'chocolate', 'vanilla', 'chocolate'],
    'Stars': [5, 3, 5, 4]
})
reviews

,Flavor,Stars
0,vanilla,5
1,chocolate,3
2,vanilla,5
3,chocolate,4


In [22]:
average_review = reviews.groupby('Flavor', as_index=False).mean()
average_review

,Flavor,Stars
0,chocolate,3.5
1,vanilla,5.0


We can join `cones` and `average_review` by providing the labels of the columns by which to join.

In [23]:
pd.merge(cones, average_review, on = 'Flavor')

,Flavor,Price,Stars
0,vanilla,4.75,5.0
1,chocolate,6.55,3.5
2,chocolate,5.75,3.5


Notice how the strawberry cones have disappeared. None of the reviews are for strawberry cones, so there is nothing to which the `strawberry` rows can be joined. This type of joining is called `Inner Join`. 

**Note:** Various joining methods are available depending on the analytical requirements. For a detailed overview of these methods and their applications, refer to the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/merging.html).

## Example: Rise of House Price with Population Growth

In this section, we will explore one of the most common use cases of join operations: finding associations between different quantities. As a data scientist, you often need to determine the relationship between variables collected from different tables or sources.

For instance, you might pose the question:
**"Does population growth impact the rise in housing prices in Canada?"**

To investigate this, let's start by displaying the two dataframes we will use for our analysis.

In [34]:
Population

,Geography,%Growth
0,Canada,1.338171
1,Newfoundland and Labrador,0.586049
2,Prince Edward Island,3.111655
3,Nova Scotia,2.064089
4,New Brunswick,1.746098
5,Quebec,0.741916
6,Ontario,1.534686
7,Manitoba,1.214651
8,Saskatchewan,0.401510
9,Alberta,1.068252


This DataFrame contains information on the population growth in different provinces in Canada for the year of 2021-2022.

In [35]:
Houseprice

,Area,y/y % change
0,Canada,21.0
1,British Columbia,23.3
2,Alberta,10.0
3,Manitoba,11.2
4,Saskatchewan,7.0
5,Ontario,25.6
6,Quebec,16.3
7,Newfoundland and Labrador,11.9
8,Nova Scotia,23.2
9,New Brunswick,32.0


This DataFrame contains information on rise in houseing prices in different provinces Canada for the year of 2021-2022.

To analyze the relationship between population growth and housing prices change, we need to combine these two DataFrames based on the common `Area` and `Geography` columns:

In [36]:
Merged = pd.merge(Population, Houseprice, left_on='Geography', right_on='Area')
Merged

,Geography,%Growth,Area,y/y % change
0,Canada,1.338171,Canada,21.0
1,Newfoundland and Labrador,0.586049,Newfoundland and Labrador,11.9
2,Prince Edward Island,3.111655,Prince Edward Island,17.6
3,Nova Scotia,2.064089,Nova Scotia,23.2
4,New Brunswick,1.746098,New Brunswick,32.0
5,Quebec,0.741916,Quebec,16.3
6,Ontario,1.534686,Ontario,25.6
7,Manitoba,1.214651,Manitoba,11.2
8,Saskatchewan,0.401510,Saskatchewan,7.0
9,Alberta,1.068252,Alberta,10.0


As Merged contains two duplicate columns, let's drop the 'Area' column.

In [38]:
Merged = Merged.drop(columns=['Area'])
Merged

,Geography,%Growth,y/y % change
0,Canada,1.338171,21.0
1,Newfoundland and Labrador,0.586049,11.9
2,Prince Edward Island,3.111655,17.6
3,Nova Scotia,2.064089,23.2
4,New Brunswick,1.746098,32.0
5,Quebec,0.741916,16.3
6,Ontario,1.534686,25.6
7,Manitoba,1.214651,11.2
8,Saskatchewan,0.401510,7.0
9,Alberta,1.068252,10.0


As we have both qualtities on the same dataframe we can proceed with our further analysis.